# Task 4 - CIFAR10 Using PyTorch

Importing required libraries and set a path for downloading dataset files

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim

path_data = './'

In this step we will download the file from internet
we can transform data when we are downloading it. The tranformation has 3 parts. first is turning PIL or numpy array data to tensors. second is normalizing data (we have to note that since first we used ToTensor function, the result would be between 0 and 1, so the mean and var for normalization is 0.5 for all channels. after taht we can download data. also we transformed the images in our dataset to one channel. since we don't wantt to try CNN at first place we made this transformation.

In [2]:
transformation = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)),
    transforms.Grayscale(num_output_channels=1)])

train_set_G = torchvision.datasets.CIFAR10(root=path_data,
                                         train=True,
                                         download=True,
                                         transform= transformation)
test_set_G = torchvision.datasets.CIFAR10(root=path_data,
                                        train=False,
                                        download=True,
                                        transform= transformation)

Files already downloaded and verified
Files already downloaded and verified


In this part with we use a function named DataLoader. this function help us iterating over Itearable datasets. we can also set number of batches for using in the model.

In [3]:
train_loader = torch.utils.data.DataLoader(train_set_G, batch_size = 5)
test_loader = torch.utils.data.DataLoader(test_set_G, batch_size = 5)

we use this function to determine the accuracy of a model over a test dataset. it will count number of correct guesses of our model over total dataset records.

In [4]:
def acc_report(model,dataset):

    total = 0
    correct = 0
    with torch.no_grad():
        for i, data in enumerate(dataset): 
            images, labels = data
            outputs = model(images)
            predicted = torch.max(outputs.data, 1)[1]
            total += labels.size()[0]
            correct += (predicted == labels).sum().item()

    print("Accuracy: ", correct/total)

## Tuning simple neural network

in this step we will create a simple NN and try to find best optimizer and learning rate for it. since the data size is big we would consider just these two factors. 

In [5]:
class NeuralNetwork(nn.Module): 
    def __init__(self): 
        super().__init__()

        self.fc1 = nn.Linear(1024, 200)
        self.fc2 = nn.Linear(200, 50)
        self.fc3 = nn.Linear(50, 10)
     
        
    def forward(self, x):
        x = x.reshape(-1, 32*32)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

M = NeuralNetwork()
Loss_func = nn.CrossEntropyLoss()

Finding best optimizer for our network:

In [8]:
optimizer1 = optim.SGD(M.parameters(), lr=0.001)
optimizer2 = optim.NAdam(M.parameters(), lr=0.001)
optimizer3 = optim.Adam(M.parameters(), lr=0.001)
optimizer4 = optim.RMSprop(M.parameters(), lr=0.001)

loss_across_epochs = []
num_epochs = 10
set_size = 10000
optimizers = [optimizer1,optimizer2,optimizer3,optimizer4]
opt_labels = ['SGD','NAdam','Adam','RMSprop']
    
for i in range(4):
    for epoch in range(num_epochs):
        train_loss= 0.0
        for data in train_loader:
            inputs, labels = data
            optimizers[i].zero_grad()
            outputs = M(inputs)  # forward pass 
            loss = Loss_func(outputs, labels)
            loss.backward()
            optimizers[i].step()
            train_loss += loss.item()
    loss_across_epochs.extend([train_loss/set_size])
    print('Accuracy for {}:'.format(opt_labels[i]))
    acc_report(M,test_loader)
    print('---------------')


Accuracy for SGD:
Accuracy:  0.4159
---------------
Accuracy for NAdam:
Accuracy:  0.4108
---------------
Accuracy for Adam:
Accuracy:  0.4019
---------------
Accuracy for RMSprop:
Accuracy:  0.3864
---------------


Finding best learning rate for our data:

In [11]:
optimizer = optim.SGD(M.parameters(), lr=0.001)

loss_across_epochs = []
num_epochs = 10
set_size = 10000
lr_list = [0.1,0.01,0.001]
    
for i in range(3):
    optimizer = optim.SGD(M.parameters(), lr=lr_list[i])
    for epoch in range(num_epochs):
        train_loss= 0.0
        for data in train_loader:
            inputs, labels = data
            optimizer.zero_grad()
            outputs = M(inputs)  # forward pass 
            loss = Loss_func(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
    loss_across_epochs.extend([train_loss/set_size])
    print('Accuracy for learning rate {}:'.format(lr_list[i]))
    acc_report(M,test_loader)
    print('---------------')



Accuracy for learning rate 0.1:
Accuracy:  0.1
---------------
Accuracy for learning rate 0.01:
Accuracy:  0.1
---------------
Accuracy for learning rate 0.001:
Accuracy:  0.1
---------------


IndexError: list index out of range

So the model best fits for SGD optimizer and learning rate of 0.1

Now we will test on a CNN network with dropouts and Maxpooling
we will load the data again, since in this stage we wan t to use all 3 channels of the image

In [16]:
transformation = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])

train_set_G = torchvision.datasets.CIFAR10(root=path_data,
                                         train=True,
                                         download=True,
                                         transform= transformation)
test_set_G = torchvision.datasets.CIFAR10(root=path_data,
                                        train=False,
                                        download=True,
                                        transform= transformation)

train_loader = torch.utils.data.DataLoader(train_set_G, batch_size = 5)
test_loader = torch.utils.data.DataLoader(test_set_G, batch_size = 5)

Files already downloaded and verified
Files already downloaded and verified


In [25]:
class NeuralNetwork(nn.Module): 
    def __init__(self): 
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5) 
        self.fc1 = nn.Linear(16*5*5, 120)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

        
        
    def forward(self, x): 
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

M = NeuralNetwork()

In [18]:
optimizer1 = optim.SGD(M.parameters(), lr=0.001)
optimizer2 = optim.NAdam(M.parameters(), lr=0.001)


loss_across_epochs = []
num_epochs = 10
set_size = 10000
optimizers = [optimizer1,optimizer2]
opt_labels = ['SGD','NAdam']
    
for i in range(2):
    for epoch in range(num_epochs):
        train_loss= 0.0
        for data in train_loader:
            inputs, labels = data
            optimizers[i].zero_grad()
            outputs = M(inputs)  # forward pass 
            loss = Loss_func(outputs, labels)
            loss.backward()
            optimizers[i].step()
            train_loss += loss.item()
    loss_across_epochs.extend([train_loss/set_size])
    print('Accuracy for {}:'.format(opt_labels[i]))
    acc_report(M,test_loader)
    print('---------------')


Accuracy for SGD:
Accuracy:  0.4789
---------------
Accuracy for NAdam:
Accuracy:  0.5617
---------------


As we can see, NAdam perforemd better in CNN this time.
now we would also try a different structure for our CNN with NAdam optimizer

In [22]:
class NeuralNetwork(nn.Module): 
    def __init__(self): 
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(32,64,3) 
        self.pool = nn.MaxPool2d(2,2)
        self.conv3 = nn.Conv2d(64,128,3)
        self.fc1 = nn.Linear(128*2*2, 128)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 10)

        
        
    def forward(self, x): 
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128*2*2)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

M = NeuralNetwork()

In [23]:
optimizer = optim.NAdam(M.parameters(), lr=0.001)
loss_across_epochs = []
num_epochs = 10
set_size = 10000

for epoch in range(num_epochs):
    train_loss= 0.0
    for data in train_loader:
        inputs, labels = data
        optimizer.zero_grad()
        outputs = M(inputs)  # forward pass 
        loss = Loss_func(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
loss_across_epochs.extend([train_loss/set_size])
print('Accuracy for new model arch')
acc_report(M,test_loader)
print('---------------')

Accuracy for new model arch
Accuracy:  0.5964
---------------


Since we performed model on our data set using different parameters, we can say simple neural networks performed weaker for our data set and using a CNN resulted in better accuracy for our dataset. Also the second CNN model which has more dropouts and more depth in first Conv2d layer was performing better than the other one. So we can say using these dropouts and also having these depth caused better performance on out model and the model was able to find better patterns in images to separate them from eachother. so, we would recommend this network. for better performance in the future.
The final CNN performance on test dataset can be seen below:


Also for more accurate result we can use 1000 epochs (since we used 10 epochs for our models to find the best one, and also dataset is very larg to perform this amount of epochs on a shoet time).

In [ ]:
# final model (to be run with a powerful device using cuda to have results in less time)

optimizer = optim.NAdam(M.parameters(), lr=0.001)
loss_across_epochs = []
num_epochs = 1000
set_size = 10000

for epoch in range(num_epochs):
    train_loss= 0.0
    for data in train_loader:
        inputs, labels = data
        optimizer.zero_grad()
        outputs = M(inputs)  # forward pass 
        loss = Loss_func(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
loss_across_epochs.extend([train_loss/set_size])
print('Accuracy for Final Model')
acc_report(M,test_loader)
print('---------------')